In [1]:
import toml
import os
import pandas as pd
import cv2

In [2]:
dict_toml = toml.load(open('../config.toml'))

FileNotFoundError: [Errno 2] No such file or directory: '../config.toml'

In [3]:
image_path = dict_toml["path"]["dataset"]["image"]

NameError: name 'dict_toml' is not defined

In [9]:
image_df = pd.read_table("../../list_attr_celeba.txt", header=0,sep=" ", index_col=0)

In [192]:
image_df["Male"]["000001.jpg"]

-1

In [11]:
f = open('../not_detected.txt', 'r')

datalist = f.read().splitlines()
f.close()

In [12]:
data = [not i in datalist for i in image_df.index]

In [13]:
image_df[data]["Male"]

000001.jpg   -1
000002.jpg   -1
000003.jpg    1
000005.jpg   -1
000007.jpg    1
             ..
202595.jpg   -1
202596.jpg    1
202597.jpg    1
202598.jpg   -1
202599.jpg   -1
Name: Male, Length: 182367, dtype: int64

In [4]:
os.listdir("../../voice/train/F/")

FileNotFoundError: [Errno 2] No such file or directory: '../../voice/train/F/'

In [ ]:
for i in os.listdir("../../voice/train/F/VCC2SF1"):
    a = pd.read_pickle("../../voice/train/F/VCC2SF1/"+i)
    print(a.shape)

# dataloader

## transform

### voice

In [155]:
class VoiceTrans(object):
    def __init__(self, maxi, mini):
        self.maxi = maxi
        self.mini = mini

    def norm_voice(self, array):
        array -= self.mini
        array /= self.maxi
    
        return array
    
    def cut(self, voice):
        return voice[:, :voice.shape[1]-voice.shape[1]%4]
    
    def __call__(self, sample):
        
        return self.cut(self.norm_voice(sample))

In [15]:
voice = pd.read_pickle("../../voice/train/VCC2SF1/10001.pkl")

In [16]:
t = VoiceTrans()
t(voice).shape

(36, 264)

### image

In [21]:
class ImageTrans(object):
    def __init__(self):
        pass

    def norm_image(self, array):
        return array/255
    
    def __call__(self, sample):
        
        return self.norm_image(sample).T

In [22]:
image = cv2.imread("../../image/000001.jpg")

In [23]:
a = ImageTrans()
a(image).shape

(3, 32, 32)

# dataset

In [121]:
import pathlib
import itertools
import pandas as pd
import numpy as np
import torch

### voice

In [168]:
class VoiceDataset(torch.utils.data.Dataset):
    def __init__(self, path, train, trans):
        p = pathlib.Path(path)
        
        if train:
            self.path = p / "train"
        else:
            self.path = p / "eval"
        
        self.trans = trans
        
        
        self.dir = [i for i in self.path.iterdir() if i.is_dir()]       
        self.file = list(itertools.chain.from_iterable([[j for j in i.iterdir()] for i in self.dir]))
        
        self.data = [pd.read_pickle(i) for i in self.file]
        self.label = [1 if "M" == str(i.parent)[-2] else -1 for i in self.file]
    
        self.datanum = len(self.data)
    
    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.label[idx]

        if self.trans:
            k = np.concatenate(c.data,axis=1)
            maxi = k.max()
            mini = k.min()
            self.transform = VoiceTrans(maxi, mini)
            
            out_data = self.transform(out_data)

        return out_data, out_label

In [169]:
c = VoiceDataset(path="../../voice", train=1, trans=1)

In [ ]:
c

### face

In [237]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, path, trans):
        p = pathlib.Path(path)
        if trans:
            self.transform = ImageTrans()
        
        self.dir = p
        self.file = [i for i in self.dir.iterdir() if i.suffix == ".jpg"]
        
        
        self.data = [cv2.imread(str(i)) for i in self.file]
        
        image_df = pd.read_table("../../list_attr_celeba.txt", header=0,sep=" ", index_col=0)
        self.label = [image_df["Male"][i.name] for i  in self.file]
        """
        for i in self.file:
            try:
                image_df["Male"][i.name]
            except:
                print(i.name)
        """     
                
        
        self.datanum = len(self.data)
    
    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.label[idx]

        if self.transform:
            
            out_data = self.transform(out_data)

        return out_data, out_label

In [238]:
f = ImageDataset("../../image", trans=1)

## dataloader

In [241]:
trainloader = torch.utils.data.DataLoader(f, batch_size = 10, shuffle = False, num_workers = 2)

In [242]:
for data,label in trainloader:
    break
print(data)
print(label)

tensor([[[[0.0039, 0.0157, 0.0039,  ..., 0.0431, 0.0471, 0.0078],
          [0.0196, 0.0039, 0.0000,  ..., 0.0431, 0.0235, 0.0196],
          [0.0039, 0.0078, 0.0157,  ..., 0.0392, 0.0196, 0.0275],
          ...,
          [0.1843, 0.1882, 0.0980,  ..., 0.0549, 0.0353, 0.0235],
          [0.1922, 0.2118, 0.1490,  ..., 0.1412, 0.0510, 0.0431],
          [0.1843, 0.2627, 0.1843,  ..., 0.1765, 0.0549, 0.0667]],

         [[0.0000, 0.0157, 0.0039,  ..., 0.0235, 0.0431, 0.0118],
          [0.0196, 0.0039, 0.0039,  ..., 0.0235, 0.0196, 0.0196],
          [0.0078, 0.0078, 0.0157,  ..., 0.0157, 0.0039, 0.0118],
          ...,
          [0.1961, 0.2118, 0.1255,  ..., 0.0235, 0.0078, 0.0039],
          [0.2118, 0.2353, 0.1922,  ..., 0.1529, 0.0588, 0.0549],
          [0.2000, 0.2902, 0.2275,  ..., 0.2078, 0.0863, 0.0941]],

         [[0.0431, 0.0706, 0.0667,  ..., 0.1294, 0.0784, 0.0039],
          [0.0667, 0.0510, 0.0588,  ..., 0.1294, 0.0549, 0.0196],
          [0.0471, 0.0549, 0.0627,  ..., 0